# Introduction 

ในการประเมิน llm โดยใช้ deepeval จะประกอบไปด้วย component หลัก 3 compoenents 
- test cases 
- metric 
- evaluation dataset

ขั้นตอน workflow การทำ evaluation 
1. จัดเตรียม evaluation dataset 
2. เขียน test case 
3. กำหนด evaluation metric 
4. execute test cases 
5. improve 

In [2]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

## Metric 

deepeval มีมากกว่า 14 metric ที่ใช้ในการประเมิน llm 

In [3]:
from deepeval.metrics import AnswerRelevancyMetric

answer_relevancy_metric = AnswerRelevancyMetric()

## Test Cases


In [4]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
  input="Who is the current president of the United States of America?",
  actual_output="Joe Biden",
  retrieval_context=["Joe Biden serves as the current president of America."]
)

In [7]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

answer_relevancy_metric = AnswerRelevancyMetric(model = azure_openai)
test_case = LLMTestCase(
  input="Who is the current president of the United States of America?",
  actual_output="Joe Biden",
  retrieval_context=["Joe Biden serves as the current president of America."]
)

answer_relevancy_metric.measure(test_case)
print(answer_relevancy_metric.score)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0


## Dataset

In [8]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.metrics import AnswerRelevancyMetric

answer_relevancy_metric = AnswerRelevancyMetric(model = azure_openai)
test_case = LLMTestCase(
  input="Who is the current president of the United States of America?",
  actual_output="Joe Biden",
  retrieval_context=["Joe Biden serves as the current president of America."]
)

dataset = EvaluationDataset(test_cases=[test_case])
dataset.evaluate([answer_relevancy_metric])

Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: Custom Azure OpenAI Model, reason: The score is 1.00 because the actual output correctly and directly addresses the question asked in the input. Great job!, error: None)

For test case:

  - input: Who is the current president of the United States of America?
  - actual output: Joe Biden
  - expected output: None
  - context: None
  - retrieval context: ['Joe Biden serves as the current president of America.']


Overall Metric Pass Rates

AnswerRelevancyMetric: 100.00% pass rate




/usr/local/lib/python3.9/site-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=True, metrics=[<deepeval.metrics.answer_relevancy.answer_relevancy.AnswerRelevancyMetric object at 0x7f1119e786d0>], input='Who is the current president of the United States of America?', actual_output='Joe Biden', expected_output=None, context=None, retrieval_context=['Joe Biden serves as the current president of America.'])]

การรันสามารถนำไปรันด้วย pytest เพื่อทำให้การ evaluate สามารถรันแบบ paralelization ได้ 

In [ ]:
from deepeval import assert_test
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

dataset = EvaluationDataset(test_cases=[...])

@pytest.mark.parametrize(
    "test_case",
    dataset,
)
def test_customer_chatbot(test_case: LLMTestCase):
    answer_relevancy_metric = AnswerRelevancyMetric()
    assert_test(test_case, [answer_relevancy_metric])